# Running open AR-Sandbox without a sandbox

The software can also be run in fully virtual mode, without an AR-Sandbox attached to it - for example to test new implementations or to evaluate the software itself, before investing time or funds to obtain an AR-Sandbox.

For this purpose, the sensor can be set to a `dummy`-sensor and a DEM can be loaded (for example from a previously saved version, or one of the DEM's supplied as additional data sets - more info below).

We show here an example with the topography-module - the method can, of course, also be combined with other modules.

In [ ]:
#Only use if sandbox is not installed
# import os,sys
# sys.path.append('../../../')

In [ ]:
# %load_ext autoreload
# %autoreload 2

## Initialize Sensor and Projector

Sensor initialisation starts out as before:

In [1]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

## Setting a `dummy`-sensor

**Note**: use `dummy`-sensor to start without actual AR-Sandbox connection: define with

```name="dummy"```

in sensor call:

In [2]:
#Import Sensor
from sandbox.sensor import Sensor
# sensor = Sensor(calibsensor=_calibsensor, name="kinect_v2")
sensor = Sensor(_calibsensor, name="dummy")
# test

sandbox.sensor.sensor_api: INFO | JSON configuration loaded for sensor.
sandbox.sensor.dummy: INFO | DummySensor initialized.


The next cell will create the window for the "projector" - in the offline version, simply open it in a separate window to follow the next steps.

In [3]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)


sandbox.projector.projector: INFO | JSON configuration loaded for projector


Launching server at http://localhost:64144
sandbox.projector.projector: INFO | Projector initialized and server started.
Please position the browser window accordingly and enter fullscreen!


## Initialize marker detection

We can also set ArUco marker positions directly by coordinates:

In [4]:
#Initialize the aruco detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor=sensor)

sandbox.markers.aruco: INFO | Using dummy arucos. Create your own aruco positions using .set_aruco_position() function
sandbox.markers.aruco: INFO | using dummy aruco module
sandbox.markers.aruco: INFO | Aruco module loaded
sandbox.markers.markers_plotting: INFO | Aruco detection ready


In [5]:
aruco.set_aruco_position(dict_position={1:[50,30],2:[100,80]}, frame=sensor.get_frame())
aruco.df_aruco_position
aruco.set_aruco_position(dict_position={}, frame=sensor.get_frame())



## Load test data


We now load topography data - here from one of the digital evlevation models supplied with open-AR-Sandbox (if the file is not found, then download the dataset with the `Download_datasets.ipynb` notebook):

In [6]:
import numpy as np

In [7]:
# file = np.load(test_data['topo'] + "DEM1.npz")
file = np.load("..\\06_LoadSaveTopoModule\saved_DEMs\DEM1.npz")
frame = file['arr_0'] # + np.min(file['arr_0'])
extent = np.asarray([0, frame.shape[1], 0, frame.shape[0], frame.min(), frame.max()])

Reset the frame position (shift to minimum value - not necessarily required, depending on the DEM):

In [8]:
frame -= np.min(frame)

## Initialize main thread for live update

Create main thread - also as in normal mode:

In [9]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco if 'aruco' in globals() else None)


sandbox.projector.contourlines: INFO | ContourLinesModule loaded successfully
sandbox.projector.shading: INFO | LightSource set to address RWTH Aachen, Germany at datetime Mon Mar 28 22:27:56 2022
sandbox.projector.cmap: INFO | CmapModule loaded successfully


Now, assing loaded frame to `main`-thread:

In [10]:
main.load_frame(frame)

sandbox.main_thread: INFO | loaded


True

In [12]:
# Start the thread 
main.run()

sandbox.main_thread: INFO | No frame to load, resuming sandbox frame acquisition


False

After this step, the loaded DEM should also appear in the projector browser tab/ window.

If you want to use the dummy live update, you can set the frame to None

```main.load_frame(None)```

### Control the features of the main thread

As in normal mode, we can now adjust settings in the projection, for example change color-map, line spacing, etc.:

In [13]:
main.widget_plot_module()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] Column
                [0] Column
                    [0] Markdown(str)
                    [1] Checkbox(name='Show colormap', value=True)
                    [2] Select(name='Choose a colormap', options=['gist_earth', ...], value='gist_earth')
                    [3] Column
                        [0] Markdown(str)
                        [1] Checkbox(name='Show relief shading', value=True)
                        [2] Tabs
                            [0] WidgetBox
                                [0] Checkbox(name='Manual configuration')
                                [1] FloatSlider(end=360.0, name='Azimuth', value=338.0111210174681, value_throttled=338.0111210174681)
                                [2] FloatSlider(end=90.0, name='Altitude', value=-33.76690482069853, value_throttled=-33.76690482069853)
                                [3] FloatInput(name='Vertical Exageration', step=0.01, value=0.25, value_throttled=0.25)
                            [1] WidgetBox
                                [0] TextInput(name='Enter address (e.g. City,..., value='RWTH Aachen, Germany')
                                [1] DatePicker(name='Select date (UTC +0)', value=datetime.date(2022, ...)
                                [2] Row(width_policy='min')
                                    [0] IntSlider(end=23, name='Hour', value=22, value_throttled=22, width_policy='min')
                                    [1] IntSlider(end=59, name='Minute', value=27, value_throttled=27, width_policy='min')
                                    [2] IntSlider(end=59, name='Second', value=56, value_throttled=56, width_policy='min')
                                [3] Markdown(str, sizing_mode='scale_width')
                                [4] Markdown(str, sizing_mode='scale_width')
                                [5] Markdown(str, sizing_mode='scale_width')
                                [6] Markdown(str, sizing_mode='scale_width')
                            [2] WidgetBox
                                [0] Checkbox(name='Start day simulation i...)
                [1] Column
                    [0] Markdown(str)
                    [1] Checkbox(name='Active contours', value=True)
                    [2] Checkbox(name='Automatic levels f...)
                    [3] Markdown(str)
                    [4] WidgetBox
                        [0] Checkbox(name='Show major contours', value=True)
                        [1] IntInput(name='Choose a contour step', value=100, value_throttled=100)
                        [2] Checkbox(name='Show minor contours', value=True)
                        [3] IntInput(name='Choose a minor c..., value=50, value_throttled=50)
                        [4] Checkbox(name='Show contours label', value=True)
                        [5] IntInput(name='set a contour l..., value=15, value_throttled=15)
            [1] Column
                [0] Column
                    [0] Markdown(str)
                    [1] Checkbox(name='Aruco Detection', value=True)
                    [2] Column
                        [0] Markdown(str)
                        [1] Row
                            [0] WidgetBox
                                [0] Checkbox(name='Show aruco location', value=True)
                                [1] Checkbox(name='Show aruco id', value=True)
                                [2] Checkbox(name='Show line connecting a...)
                                [3] ColorPicker(name='Color', value='#ff0000')
                            [1] WidgetBox
                                [0] Markdown(str)
                                [1] Row
                                    [0] IntInput(name='id', value=1, value_throttled=1, width=70)
                                    [1] IntInput(name='x coordinate', value=50, value_throttled=50, width=70)
                                    [2] IntInput(name='y coordinate', value=50, value_throttled=50, width=70)
         

For further use, we can also save the generated projector figure (for example, to generate teaching material, visualization in external software, etc.):

In [14]:
# test saving figure
if True:
    projector.figure.savefig("terrain_plot.png")
    # projector.figure.axes[0].plot([50,100],[50,100], 'k-')
    # projector.figure.axes[0].contourf(frame)

## Import the desired module

In [15]:
# Import the modules to use
from sandbox.modules import TopoModule


In [16]:
TopoModule = TopoModule(extent=sensor.extent)

sandbox.modules.topography: INFO | TopoModule loaded successfully


## Add the module to the main thread

In [21]:
main.add_module(name ='Topo', module=TopoModule)

sandbox.main_thread: INFO | module Topo added to modules


In [23]:
TopoModule.sea_level_polygon_alpha = 0.6

To remove the module you can type 

```main.remove_module('Topo')```

## Use widgets to modify the parameters

In [20]:
##### Topography Widgets
TopoModule.show_widgets()

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] IntInput(name='Minimum height o..., step=20, value=-200, value_throttled=-200)
            [1] IntInput(name='Maximum height o..., step=20, value=800, value_throttled=800)
            [2] Checkbox(name='Show sea level')
            [3] Row
                [0] Checkbox(name='Show sea level contour', width_policy='min')
                [1] Checkbox(name='Fill contour', value=True, width_policy='min')
            [4] IntSlider(end=800, name='Set sea level height', start=-200, step=5, value_throttled=0)
        [1] Column
            [0] Checkbox(name='Animate wave movement', value=True)
            [1] Select(options=['water', 'lava', ...], size=3, value='water')
            [2] FloatInput(name='Select transparency', step=0.05, value=0.7, value_throttled=0.7)
            [3] ColorPicker(name='Color level contour', value='#0000ff')
    [2] Markdown(str)
    [3] Checkbox(name='Local level')

In [24]:
#### Stop the thread
main.stop()

sandbox.main_thread: INFO | Thread stopped.
